*эта часть с сайта naturalantibody, тут через pyspark делают, это не очень удобно

# Antibody specific database usage examples

## Downloading the data (takes 1-8 minutes depnending on location of the runtime machine)
This part of the code consiss of listing and downloading each part of the Antigen Specific Database to your local runtime.


In [ ]:
!pip install -U PyDrive2 -qq

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
ID = "1ZhACiqV8_JvyLQ8iUCSaTtf1BvhoeDHB"

# Download all files recursively
def download_folder(folder_id, save_path):
    os.makedirs(save_path, exist_ok=True)
    file_list = drive.ListFile({
        'q': f"'{folder_id}' in parents and trashed=false"
    }).GetList()

    for file in file_list:
        file_path = os.path.join(save_path, file['title'])
        if file['mimeType'] == 'application/vnd.google-apps.folder':
            download_folder(file['id'], file_path)
        else:
            # print(f'Downloading {file["title"]}...')
            file.GetContentFile(file_path)

# Start download
download_folder(ID, '/content/drive/MyDrive/Antibody_Chekalin_Denis/asd')

## Installing Dependencies - takes 2-3 minutes
In order to run the code, it is necesseary to install corret java and spark version along with the required libraries:
* Pyspark - for reading and proessing the data
* delta spark - for reading the data in source (delta) format


In [ ]:
!sudo apt update
!sudo apt install openjdk-8-jdk-headless
!export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64
!sudo apt-get install awscli
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3-scala2.13.tgz
!tar xf spark-3.5.6-bin-hadoop3-scala2.13.tgz
!pip install -q findspark
!pip install "pyspark<4.0"
!pip install "delta-spark<4.0"
!pip install py4j
!pip install json2schema
# !pip install 'urllib3<2'

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as re

### Initialize Spark

In [ ]:
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[2] pyspark-shell"

import sys
from delta import *

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

from contextlib import contextmanager
from typing import Dict, Iterator, List, Optional

from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession

from google.colab import data_table

builder = SparkSession \
  .builder \
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark

In [ ]:
import json
from pathlib import Path
from tempfile import TemporaryDirectory

def show_as_json(df: DataFrame) -> None:
    with TemporaryDirectory() as tmpdir:
        output_path = Path(tmpdir) / "tmp.jsonl"
        df.limit(1).coalesce(1).write.format('json').save(str(output_path))
        single_file = list(output_path.glob("*.json"))[0]
        with open(str(single_file)) as f:
            print(json.dumps(json.loads(f.read()), indent=4))

## Loading the dataset

In [ ]:
df = spark.read.format('delta').load('/content/drive/MyDrive/Antibody_Chekalin_Denis/asd')
df = df.orderBy("dataset")
df.show()
df.printSchema()

+-------+--------------------+--------------+-----+-------------+--------+--------------------+----------+--------+--------------------+---------------------+
|dataset|      heavy_sequence|light_sequence| scfv|affinity_type|affinity|    antigen_sequence|confidence|nanobody|            metadata|processed_measurement|
+-------+--------------------+--------------+-----+-------------+--------+--------------------+----------+--------+--------------------+---------------------+
|    aae|QVQLQQSGAELVRPGAS...|              |false|           kd|    0.66|KVFGRCELAAAMKRHGL...| very_high|   false|{hen egg-white ly...|                 0.66|
|    aae|QVQLQQSGAELVRPGAS...|              |false|           kd|   0.872|KVFGRCELAAAMKRHGL...| very_high|   false|{hen egg-white ly...|                0.872|
|    aae|QVQLQQSGAELVRPGAS...|              |false|           kd|   0.854|KVFGRCELAAAMKRHGL...| very_high|   false|{hen egg-white ly...|                0.854|
|    aae|QVQLQQSGAELVRPGAS...|              |f

In [ ]:
print(f'Всего записей: {df.count()}')

print("\nКоличество записей для каждого датасета:")
dataset_counts = df.groupBy("dataset").count().orderBy("dataset")
dataset_counts.show(truncate=False)

Всего записей: 1227083

Количество записей для каждого датасета:
+-----------------------+------+
|dataset                |count |
+-----------------------+------+
|aae                    |35    |
|aatp                   |93    |
|ab-bind                |283   |
|abbd                   |155853|
|abdesign               |672   |
|alphaseq               |198703|
|biomap                 |2725  |
|buzz                   |524346|
|covid-19               |54625 |
|dlgo                   |360   |
|flab_hie2022           |55    |
|flab_koenig2017        |4275  |
|flab_rosace2023        |25    |
|flab_shanehsazzadeh2023|446   |
|flab_warszawski2019    |2048  |
|genbank                |2989  |
|hiv                    |48008 |
|literature             |5636  |
|met                    |4000  |
|osh                    |30    |
+-----------------------+------+
only showing top 20 rows



In [ ]:
df = df.drop("metadata") # we drop it to be able to save the data to csv, if you can sue parquet it is not necesseary

### Exemplary EDA
This part explores the dataset, ways to save and load selected sub-selections.

In [ ]:
from pyspark.sql.functions import col, concat_ws, countDistinct
import pandas as pd

# Extract all available datasets
datasets = df.select("dataset").distinct().collect()
dataset_list = [row.dataset for row in datasets if row.dataset is not None]

print("Avaliable Datasets:")
for dataset in dataset_list:
  print(f"\t- {dataset}")

#### Saving/loading specific datasets

In [ ]:
from pyspark.sql import DataFrame, SparkSession
import os

def save_as_csv(df: DataFrame, path: str, mode: str = "overwrite", header: bool = True):
    df.write.mode(mode).option("header", header).csv(path)

def load_csv(spark: SparkSession, path: str, header: bool = True, inferSchema: bool = True) -> DataFrame:
    return spark.read.option("header", header).option("inferSchema", inferSchema).csv(path)

def save_as_parquet(df: DataFrame, path: str, mode: str = "overwrite"):
    df.write.mode(mode).parquet(path)

def load_parquet(spark: SparkSession, path: str) -> DataFrame:
    return spark.read.parquet(path)

chosen_datasets = ["structures-nanobodies", " structures-antibodies"]
selected_df = df.filter(col("dataset").isin(chosen_datasets))

# Save to CSV
csv_path = "/tmp/selected_df_csv"
save_as_csv(selected_df, csv_path)

# Load from CSV
loaded_csv_df = load_csv(spark, csv_path)
print("Displaying results for loading csv...")
loaded_csv_df.show(1, truncate=False, vertical=True)

# Save to Parquet
parquet_path = "/tmp/selected_df_parquet"
save_as_parquet(selected_df, parquet_path)

# Load from Parquet
print("Displaying results for loading Parquet...")
loaded_parquet_df = load_parquet(spark, parquet_path)
loaded_parquet_df.show(1, truncate=False, vertical=True)


Displaying results for loading csv...
-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------
 dataset               | structures-nanobodies                                                                                                                          
 heavy_sequence        | EVQLVESGGRLVRTGGSLRLSCAASGRTFSNYVMGWFRQAPGKEREVVAAITWSGDITWHADFVKGRFTISRDNAKNTVYLQMNSLKPEDTAVYYCAATERWGLRAPADWGSWGQGTQVTVSSHGSGLVPRGSGGGHHHHHH 
 light_sequence        | NULL                                                                                                                                           
 scfv                  | false                                                                                                                                          
 affinity_type         | bool                                                                                        

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number

# Define columns for uniqueness
unique_cols = ["heavy_sequence", "light_sequence", "antigen_sequence"]

# Create row_number window for deduplication
window_spec = Window.partitionBy(*unique_cols).orderBy("dataset")  # You can change ordering

# === NANOBODY ===
nanobody_df = df.filter(col("nanobody") == True)

nanobody_unique = nanobody_df.withColumn("row_num", row_number().over(window_spec)) \
                             .filter(col("row_num") == 1) \
                             .drop("row_num")

# Save
nanobody_path = "/tmp/nanobody_unique.csv"
save_as_csv(nanobody_unique, nanobody_path)

# Load
loaded_nanobody_unique = load_csv(spark, nanobody_path)

print("=== Unique Nanobody Record ===")
loaded_nanobody_unique.show(1, truncate=False, vertical=True)

# === SCFV ===
scfv_df = df.filter(col("scfv") == True)

scfv_unique = scfv_df.withColumn("row_num", row_number().over(window_spec)) \
                     .filter(col("row_num") == 1) \
                     .drop("row_num")

# Save
scfv_path = "/tmp/scfv_unique.csv"
save_as_csv(scfv_unique, scfv_path)

# Load
loaded_scfv_unique = load_csv(spark, scfv_path)

print("=== Unique scFv Record ===")
loaded_scfv_unique.show(1, truncate=False, vertical=True)


In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number

# Define uniqueness criteria
unique_cols = ["heavy_sequence", "light_sequence", "antigen_sequence"]
window_spec = Window.partitionBy(*unique_cols).orderBy("dataset")

# === 1. Binary affinity_type (affinity_type == "bool") ===
binary_df = df.filter(col("affinity_type") == "bool")

binary_unique = binary_df.withColumn("row_num", row_number().over(window_spec)) \
                         .filter(col("row_num") == 1) \
                         .drop("row_num")

# Save
binary_path = "/tmp/binary_affinity_unique.csv"
save_as_csv(binary_unique, binary_path)

# Load
loaded_binary_unique = load_csv(spark, binary_path)

print("=== Unique Binary Affinity Record ===")
loaded_binary_unique.show(1, truncate=False, vertical=True)

# === 2. Non-binary affinity_type (affinity_type != "bool") ===
non_binary_df = df.filter((col("affinity_type").isNotNull()) & (col("affinity_type") != "bool"))

non_binary_unique = non_binary_df.withColumn("row_num", row_number().over(window_spec)) \
                                 .filter(col("row_num") == 1) \
                                 .drop("row_num")

# Save
non_binary_path = "/tmp/non_binary_affinity_unique.csv"
save_as_csv(non_binary_unique, non_binary_path)

# Load
loaded_non_binary_unique = load_csv(spark, non_binary_path)

print("=== Unique Non-Binary Affinity Record ===")
loaded_non_binary_unique.show(1, truncate=False, vertical=True)


=== Unique Binary Affinity Record ===
-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 dataset               | patents                                                                                                                                                                                                  